# Audiolibros en español

In [ ]:
# !pip install pydub
# !pip install num2words
# !pip install fairseq sentencepiece
# !pip install huggingface_hub
# !pip install pdfplumber
# !pip install git+https://github.com/openai/whisper.git
# !pip install Levenshtein

In [19]:
from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_mp3(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="mp3")


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
print(os.environ['TF_ENABLE_ONEDNN_OPTS'])  # Salida: mi_valor

0


In [3]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj
    

# Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]

# Movemos el modelo al dispositivo GPU
model = model.to('cuda:0')

# Actualizamos la configuración con los datos del task
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Creamos el generador
generator = task.build_generator([model], cfg)


# Texto a convertir en voz
text = "Hola, esta es una prueba. Quiero saber hasta dónde llega el modelo."

# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


2023-09-07 11:33:30.407163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 11:33:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2023-09-07 11:33:34 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 107
2023-09-07 11:33:34 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
2023-09-07 11:33:35 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin


In [4]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [5]:
# Diccionario con las traducciones
translations = {
    'Dr': 'Doctor',
    'Sr': 'Señor',
    'Sra': 'Señora',
    # Añade más traducciones aquí
}

def process_abrev(line):
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'


El Doctor está hablando con el Señor Pérez y la Señora Gómez.
Buenos días Señor Gómez.


In [6]:
# Diccionario con las traducciones
translations = {
    '-': ',',
    '—': ',' 
    # Añade más traducciones aquí
}

def otras_traducciones(line):
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'


Hola ,cómo estás,?
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [7]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

In [8]:
# Texto a convertir en voz
text = "Hola, esta es una prueba. ¿Está de acuerdo Sr. López? Voy a comprobar cuántas frases puede decir de golpe de 25 palabras."
text = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

text = preprocesado_al_modelo(text)
print(text)
# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio

El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [9]:
def spanish_t2s(text):
    text = preprocesado_al_modelo(text)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, text)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    return wav, rate

# Parsear PDF y preparar los mini-parrafos

In [ ]:
import re
import pdfplumber

parrafos = []
word_count = 0
current_paragraph = ""

def leer_pdf(ruta_pdf, pagina_inicio=1):
    with pdfplumber.open(ruta_pdf) as archivo_pdf:
        texto_completo = ""
        for pagina in archivo_pdf.pages[pagina_inicio:]:
            texto_completo += pagina.extract_text()
    return texto_completo

def procesar_texto(texto):
    global word_count
    global current_paragraph
    global parrafos

    # Eliminar saltos de línea que no sean líneas en blanco que separen párrafos. También eliminar los retornos de carro.
    texto_procesado = re.sub(r'(?<!\n)\n(?!\n)', ' ', texto).replace('\r', '')
    
    # Iterar sobre cada carácter del texto
    for char in texto_procesado:
        current_paragraph += char  # Añadir el carácter al párrafo actual

        if char == ' ':  # Contar las palabras
            word_count += 1

        # Dividir el párrafo si se encuentra con un signo de puntuación y hay más de 12 palabras
        if char in ['.', ',', ':', ';', '?', '!', '-', '—'] and word_count >= 13:
            parrafos.append(current_paragraph.strip())
            current_paragraph = ""
            word_count = 0

    # Añadir el último párrafo si contiene texto
    if current_paragraph:
        parrafos.append(current_paragraph.strip())

pdf_path = './El_Senor_de_los_Anillos_J_R_R_Tolkien.pdf'
texto = procesar_texto(leer_pdf(pdf_path))

# Imprimir los párrafos generados
for i, para in enumerate(parrafos):
    print(f"Paragraph {i + 1}:\n{para}\n")

In [ ]:
for i, para in enumerate(parrafos):
    print(f"Paragraph {i + 1}:\n{para}\n")
    # si texto contiene a "libro trata principalmente de los Hobbits" salir del bucle
    if "libro trata principalmente de los Hobbits" in para:
        break

    

In [30]:
# para construir un pequeño ejemplo de audio con la primeras 30 frases
parrafos = parrafos[17:50]

In [ ]:
for i, para in enumerate(parrafos):
    print(f"Paragraph {i + 1}:\n{para}\n")
    wav, rate = spanish_t2s(para)
    ipd.display(ipd.Audio(wav.cpu(), rate=rate))
parrafos

# Creamos audio

In [ ]:
import whisper
modelWhisper = whisper.load_model('medium')

In [35]:
import Levenshtein

texto1= "hola como estas"
texto2= "hola, como estas?"
# funcion que quita simbolos de puntuacion
def quitar_puntuacion(texto):
    texto = texto.replace('.', '')
    texto = texto.replace(',', '')
    texto = texto.replace('?', '')
    texto = texto.replace('!', '')
    texto = texto.replace('¿', '')
    texto = texto.replace('¡', '')
    texto = texto.replace(';', '')
    texto = texto.replace(':', '')
    texto = texto.replace('(', '')
    texto = texto.replace(')', '')
    texto = texto.replace('[', '')
    texto = texto.replace(']', '')
    texto = texto.replace('{', '')
    texto = texto.replace('}', '')
    texto = texto.replace('"', '')
    texto = texto.replace("'", '')
    texto = texto.replace('`', '')
    texto = texto.replace('´', '')
    texto = texto.replace('’', '')
    texto = texto.replace('‘', '')
    texto = texto.replace('“', '')
    texto = texto.replace('”', '')
    texto = texto.replace('…', '')
    texto = texto.replace('«', '')
    texto = texto.replace('»', '')
    texto = texto.replace('–', '')
    texto = texto.replace('—', '')
    texto = texto.replace('−', '')
    texto = texto.replace('‐', '')
    texto = texto.replace('⁃', '')
    texto = texto.replace('‒', '')
    texto = texto.replace('―', '')
    texto = texto.replace('‹', '')
    texto = texto.replace('›', '')
    texto = texto.replace('•', '')
    texto = texto.replace('·', '')
    texto = texto.replace('‣', '')
    texto = texto.replace('⁌', '')
    texto = texto
    return texto

# funcion que calcula la distancia entre dos textos
Levenshtein.ratio(quitar_puntuacion(texto1), quitar_puntuacion(texto2))

1.0

In [ ]:
import os
import torchaudio
import Levenshtein
import shutil


small_temp_files = []

# crear carpeta temporal si no existe
carpetaTemporal = "./asudioLibro_sp"
if not os.path.exists(carpetaTemporal):
    os.makedirs(carpetaTemporal)

# Itera sobre cada párrafo
for i, text in enumerate(parrafos):
    print(f"Paragraph {i + 1}:\n{text}\n")


    text = parrafos[i]
    text = preprocesado_al_modelo(text)

    # text = "pues, contaba el viaje de Bilbo hacia el este y la vuelta,"
    print(f"Paragraph {i + 1}:\n{text}\n")
    # Genera el audio

    wav, rate = spanish_t2s(text)

    if len(wav.shape) == 1:
        wav = wav.unsqueeze(0)

    temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
    torchaudio.save(temp_file_name, wav.to('cpu'), rate)

    similitud = 0
    contador = 0
    line = text
    longitud_texto = len(text.split())
    while True:
        
        prediction = modelWhisper.transcribe(temp_file_name, language="Spanish", temperature=0, beam_size=1)["text"]
        similitud = Levenshtein.ratio(quitar_puntuacion(line), quitar_puntuacion(prediction))
        print("Linea: ", line)
        print("Perdiction: ", prediction)
        print("Similitud: ", similitud)
        print("Contador: ", contador)
        

        print("Provisional transcripted_sentence:", prediction)
        if (similitud > 0.96) or (similitud > 0.93 and longitud_texto < 24) or (similitud > 0.90 and contador > 8) or (similitud > 0.88 and contador > 11) or (similitud > 0.85 and contador > 15) or contador > 17:
            break
        else:
            contador += 1
            if contador > 2 and longitud_texto < 24 or contador > 4:
                line = " ".join(text.split()[:contador]) + "," + " ".join(text.split()[contador:])
            wav, rate = spanish_t2s(line)
            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)

    temp_file_name_definitivo = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.mp3"
    # convertir .wav en .mp3
    !ffmpeg -y -i "{temp_file_name}" "{temp_file_name_definitivo}"
    # eliminar .wav
    !rm "{temp_file_name}"
    # añadir fichero a la lista de ficheros temporales
    small_temp_files.append(temp_file_name_definitivo)


# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "{carpetaTemporal}/audioLibro_output.mp3"

# quitamos silencios
quitar_silencios(f"{carpetaTemporal}/audioLibro_output.mp3", f"{carpetaTemporal}/audioLibro_output_sin_silencios.mp3")
!cp "{carpetaTemporal}/audioLibro_output_sin_silencios.mp3" "./audioLibro_sp_output_sin_silencios.mp3"
shutil.rmtree(carpetaTemporal)

In [38]:
# import shutil

# # quitamos silencios
# quitar_silencios(f"{carpetaTemporal}/audioLibro_output.mp3", f"{carpetaTemporal}/audioLibro_output_sin_silencios.mp3")
# !cp "{carpetaTemporal}/audioLibro_output_sin_silencios.mp3" "./audioLibro_sp_output_sin_silencios.mp3"
# shutil.rmtree(carpetaTemporal)

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


# Pruebas

In [ ]:
parrafos

In [12]:
import whisper
modelWhisper = whisper.load_model('medium')

In [16]:
# Utilizamos whisper (modelo con muy alto accuracy) para evaluar la calidad del audio generado.
import whisper
import Levenshtein
import torchaudio

carpetaTemporal = "./asudioLibro_sp"

i=4
text = parrafos[i]
# text = "pues, contaba el viaje de Bilbo hacia el este y la vuelta,"
print(f"Paragraph {i + 1}:\n{text}\n")
# Genera el audio

wav, rate = spanish_t2s(text)

if len(wav.shape) == 1:
    wav = wav.unsqueeze(0)

temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
torchaudio.save(temp_file_name, wav.to('cpu'), rate)

similitud = 0
contador = 0
line = text
while True:
    
    prediction = modelWhisper.transcribe(temp_file_name, language="Spanish", temperature=0, beam_size=1)["text"]
    similitud = Levenshtein.ratio(line, prediction)
    print("Linea: ", line)
    print("Perdiction: ", prediction)
    print("Similitud: ", similitud)
    print("Contador: ", contador)
    

    print("Provisional transcripted_sentence:", prediction)
    if (similitud > 0.93) or (similitud > 0.90 and contador > 7) or (similitud > 0.88 and contador > 10) or contador > 13:
        break
    else:
        contador += 1
        if contador > 2:
            line = " ".join(text.split()[:contador]) + "," + " ".join(text.split()[contador:])
        wav, rate = spanish_t2s(line)
        if len(wav.shape) == 1:
            wav = wav.unsqueeze(0)
        torchaudio.save(temp_file_name, wav.to('cpu'), rate)

# convertir .wav en .mp3
!ffmpeg -y -i "{temp_file_name}" "{carpetaTemporal}/temp_{str(i).zfill(5)}.mp3"

Paragraph 5:
pues contaba el viaje de Bilbo hacia el este y la vuelta,



Linea:  pues contaba el viaje de Bilbo hacia el este y la vuelta,
Perdiction:   pues contaba el viaje de Bilbo hacia el este y la vuelta.
Similitud:  0.9739130434782609
Contador:  0
Provisional transcripted_sentence:  pues contaba el viaje de Bilbo hacia el este y la vuelta.


In [69]:
import os
import torchaudio
import whisper
import Levenshtein


# Utilizamos whisper (modelo con muy alto accuracy) para evaluar la calidad del audio generado.
model = whisper.load_model('large-v2')

small_temp_files = []

# crear carpeta temporal si no existe
carpetaTemporal = "./asudioLibro_sp"
if not os.path.exists(carpetaTemporal):
    os.makedirs(carpetaTemporal)

# Itera sobre cada párrafo
for i, text in enumerate(parrafos):
    print(f"Paragraph {i + 1}:\n{text}\n")
    # Genera el audio
    wav, rate = spanish_t2s(text)

    prediction = model.transcribe(wav, language="Spanish", temperature=0, beam_size=1)["text"]
    similitud = Levenshtein.ratio(text, prediction)
    print("Perdictio: ", prediction)
    print("Similitud: ", similitud)

    if len(wav.shape) == 1:
        wav = wav.unsqueeze(0)

    temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.mp3"
    torchaudio.save(temp_file_name, wav.to('cpu'), rate)
    small_temp_files.append(temp_file_name)

# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "{carpetaTemporal}/audioLibro_output.mp3"

# import shutil
# shutil.rmtree(carpetaTemporal)

: 

In [ ]:
# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "{carpetaTemporal}/final_output.mp3"